### Imports

In [80]:
# imports
import numpy as np
from astropy import units as u
from astropy import coordinates as c
from astropy.cosmology import units as cu
import matplotlib.pyplot as plt
from matplotlib import rcParams as P
from scipy.spatial.transform import Rotation as R
import math
import random


### Helper functions

#### angle generator

In [82]:
def generate_angles():
    # generates the 3 needed euler angles at random
    # angles
    alpha = 2 * np.pi * np.random.random()  #generate random rotation angles
    beta = np.pi * np.random.random()
    gamma = 2 * np.pi * np.random.random()
    
    # alpha, beta, gamma = 0.1, 0, 0
    
    return (alpha, beta, gamma)

#### galaxy rotator

In [83]:
# This function performs an Euler rotation on the data, returns the new glon and glat
# prof. watkins' code (plus some additions of my own)
# uses scipy to preform a random euler rotation  
def rotate(array, angles):
    
    # 'angles' tuple must have 3 elements that coorespons to the euler angles of rotation
    alpha, beta, gamma = angles
    
    # data part
    glon = array[10] * np.pi / 180
    glat = array[11] * np.pi / 180

    #make glon and glat into array of positions on the unit sphere
    pos = np.array([np.cos(glon) * np.cos(glat),  # alpha
                    np.sin(glon) * np.cos(glat),  # beta
                    np.sin(glat)])                # gamma

    pos = np.transpose(pos)

    r = R.from_euler('xzx', [alpha, beta, gamma]) #create the rotation
    posprime = r.apply(pos)  #apply the rotation to the vectors

    glonprime = np.arctan2(posprime[1], posprime[0]) * 180 / np.pi  #convert back to degrees
    # glonprime[glonprime < 0] = 360 + glonprime[glonprime < 0]
    if glonprime < 0:
        glonprime += 360

    glatprime = np.arcsin(posprime[2]) * 180/np.pi
    
    # glonprime, glatprime = c.Angle(glonprime, u.degree), c.Angle(glatprime, u.degree)
    glonprime = c.Angle(glonprime, u.degree)
    glatprime = c.Angle(glatprime, u.degree)
    
    # print(f'{alpha=}, {beta=}, {gamma=}')  # angles of rotation (radians)
    glonprime, glatprime = glonprime.wrap_at('180d', inplace=False).radian, glatprime.radian
    
    return glonprime, glatprime

#### bulkflow rotator

In [84]:
def rotate_bulkflow(bulkflow, angles: tuple):
    # rotates one bulkflow for ONE catalogue at a time, to be used in a loop in sync with the previous rotate function
    
    # 'angles' tuple must have 3 elements that coorespons to the euler angles of rotation
    alpha, beta, gamma = angles
    
    # rotation part
    r = R.from_euler('xzx', [alpha, beta, gamma])
    bulkflowprime = r.apply(bulkflow)
        
    return bulkflowprime  # returns an array of the new components of the new bulkflow

#### distance modulus 

In [85]:
# evaluating distance mod
def distance_mod(line, error):
    distance = line[7]
    return 5 * math.log10(distance) + 25 + error

#### Loading, processing, rotating the data

In [86]:
bulkflows = np.load('C:/Users/yaras/Documents/Research/Feldman/outerrim/CF3-OuterRim-CF3grouplike-cz-rand/true_bulkflows100.npy')

# load the data
for i in range(1): # use range(300) to run on all catalogues (BAD IDEA)
    data = np.load(f'C:/Users/yaras/Documents/Research/Feldman/outerrim/CF3-OuterRim-CF3grouplike-cz-rand/CF3-OuterRim-CF3grouplike-cz-rand-box-{i}.npy')
    for v in range(3):

        # all arrays will have size 11132x7
        new_cat = np.empty((11132, 7))
        
        # accounting for zone of avoidance
        indices = np.where(abs(data[:,11]) > 5)
        data = data[indices]
        np.random.shuffle(data)
        data = data[:11132]  # 11132 is the size of the smallest catalogue
        

        # generating multiple versions of each catalogue
        for line in data:
            '''
            generating angles of rotation
            'angles' will be a tuple with 3 elements (alpha, beta, gamma)
            look cell No. 4 (generate angles) for function definition
            '''
            angles = generate_angles()
            
            
            '''
            rotating the bulkflows
            'bulkflow' will be a 1x3 array with the xyz components
            of the bulkflow for the respective catalogue
            look cell No. 6 (bulkflow rotator) for function definition
            '''
            bulkflow = bulkflows[:,i]
            bulkflow_prime = rotate_bulkflow(bulkflow, angles)
            
            '''
            euler rotation of the data using the generated angles
            '''
            glonprime, glatprime = rotate(line, angles)
            
            '''
            generating an error between 0 and 40% on a galcian distribution
            '''
            error = np.random.normal() * 0.4
            # error = 0  # uncomment this to set error to 0
            
            '''
            distance modulus
            '''
            d_mod = distance_mod(line, error)
            
            '''
            redshift
            '''
            redshfit = line[6]
            
            '''
            writing the new line
            '''
            new_array = np.array([redshfit,   # redshift
                                d_mod,      # distance modulus
                                glonprime,  # galactic longitude prime
                                glatprime,  # galactic latitude prime
                                bulkflow_prime[0],   # bulkflow x
                                bulkflow_prime[1],   # bulkflow y
                                bulkflow_prime[2]])  # bulkflow z
            
            np.append(new_cat, [new_array], axis=0)
            
        # with open(f'C:/Users/yaras/Documents/Research/Feldman/rotated-outerrim-cz-rand/rotated-{i}-version-{v}.npy', 'w') as fhand:
        #     for line in new_cat:
        #         for element in line:
        #             fhand.write(f'{element}')
        #         fhand.write()
        np.save(f'C:/Users/yaras/Documents/Research/Feldman/rotated-outerrim-cz-rand/rotated-{i}-version-{v}.npy', new_cat)
            
    


    

#### visualizing the data

In [ ]:
# for generating figures
# for i in range(1):
# C:/Users/yaras/Documents/Research/Feldman/outerrim/CF3-OuterRim-CF3grouplike-cz-rand/CF3-OuterRim-CF3grouplike-cz-rand-box-{i}.npy
#     data = np.load(f'INCLUDE PATH HERE')

    # # processing the ORIGINAL data, we dont need this for making the new catalogues
    # # ------------------------------------
    # # glat and glon are in degrees, make unit objects to ease operations
    # glon, glat = data[: ,10], data[: ,11]
    # glon = c.Angle(glon, u.degree)
    # glat = c.Angle(glat, u.degree)
                
    # # changing range of glon to (-180, 180) instead of (0, 360)
    # glon = glon.wrap_at('180d', inplace=False)

    # # Accounting for zone of avoidance (taking out data points with -5 < glat < 5)
    # indices = np.where(abs(glat.value) > 5)
    # glon = glon[indices]
    # glat = glat[indices]
    # # -------------------------------------

    # # making figure
    # fig = plt.figure(figsize=(20, 20))
    # P.update({'font.size': 18,
    #                    'axes.linewidth': 1.5,
    #                    'axes.labelweight': 'bold',
    #                    })

    # # subplot 1
    # ax1 = fig.add_subplot(221, projection='hammer')
    # ax1.scatter(glon.radian, glat.radian, marker='.', linewidths=0.001)
    # ax1.scatter(glonprime, glatprime, marker='.', linewidths=0.001, color='red')
    # ax1.grid(True)
    # ax1.title.set_text(f'No. {i}')

    # the angles will be printed
    # print('original bulkflow xyz:\n', bulkflows[:,i])
    # print('rotated bulkflow xyz:\n', bulkflow_prime)
    # print('original bulkflow magnitude:\n', math.sqrt(bulkflows[:,i][0]**2+bulkflows[:,i][1]**2+bulkflows[:,i][2]**2))
    # print(f'rotated bulkflow magnitude (with {error=}):\n', math.sqrt(bulkflow_prime[0]**2+bulkflow_prime[1]**2+bulkflow_prime[2]**2))

    # # subplot 2
    # ax2 = fig.add_subplot(222, projection='hammer')
    # ax2.scatter(glonprime, glatprime, marker='.', linewidths=0.001, color='red')
        
    # ax2.grid(True)
    # ax2.title.set_text(f'No. {i}')

    # x,y,z = bulkflows[:,i]
    # x1,y1,z1 = bulkflow_prime

    # ax = fig.add_subplot(223, projection='3d')
    # ax.plot((0, x) ,(0, y),(0, z))
    # ax.plot((0, x1) ,(0, y1),(0, z1))
    # ax.scatter(x,y,z)
    # ax.scatter(x1,y1,z1)
    # ax.view_init(-155, 45)

    # fig.show()

#### misc

In [87]:
# print(data)
# np.random.shuffle(data)
# print(data)
# print(len(data[:11132]))

In [88]:
# this cell helped me find the smallest and largest catalogues
print(max([len(np.load(f'C:/Users/yaras/Documents/Research/Feldman/outerrim/CF3-OuterRim-CF3grouplike-cz-rand/CF3-OuterRim-CF3grouplike-cz-rand-box-{i}.npy')) for i in range(300)]))
print(min([len(np.load(f'C:/Users/yaras/Documents/Research/Feldman/outerrim/CF3-OuterRim-CF3grouplike-cz-rand/CF3-OuterRim-CF3grouplike-cz-rand-box-{i}.npy')) for i in range(300)]))
# max: 11809
# min: 11132

11809
11132
